In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# CUDA Code

In [10]:
%%writefile cuda.cu
#include <iostream>
#include <cmath>
#include <stdio.h>

#define MAX_SHARED_MEM 2048
#define MAX_BLOCK_SIZE 1024

using namespace std;

struct Matrix {
    int rows;
    int cols;
    double *mat;
};

double getElement(Matrix m, int row, int col) {
    return m.mat[row * m.cols + col];
}

void setElement(Matrix m, int row, int col, double val) {
    m.mat[row * m.cols + col] = val;
}

__device__ double getElementDev(Matrix m, int row, int col) {
    return m.mat[row * m.cols + col];
}

__device__ void setElementDev(Matrix m, int row, int col, double val) {
    m.mat[row * m.cols + col] = val;
}

void allocateMatrix(Matrix &m, int size) {
    m.rows = size;
    m.cols = size * 2;
    m.mat = new double[m.rows * m.cols];
    for(int i = 0; i < size; ++i) {
        for(int j = 0; j < 2*size; ++j) {
            setElement(m, i, j, (j - size == i) ? 1.0 : 0.0);
        }
    }
}

void readMatrix(Matrix &m) {
    int n;
    cin >> n;
    allocateMatrix(m, n);
    for (int i = 0; i < n; ++i)
        for (int j = 0; j < n; ++j){
            double x;
            cin >> x;
            setElement(m, i, j, x);
        }
}

void printMatrix(const Matrix &m) {
    for(int i = 0; i < m.rows; ++i) {
        for(int j = 0; j < m.cols; ++j) {
            printf("%lf ", getElement(m, i, j));
        }
        printf("\n");
    }
}

void printIdent(const Matrix &m) {
    for(int i = 0; i < m.rows; ++i) {
        for(int j = m.rows; j < m.cols; ++j) {
            printf("%lf ", getElement(m, i, j));
        }
        printf("\n");
    }
}

__device__ void printMatrixDev(const Matrix &m) {
    for(int i = 0; i < m.rows; ++i) {
        for(int j = 0; j < m.cols; ++j) {
            printf("%lf ", getElementDev(m, i, j));
        }
        printf("\n");
    }
}

__device__ int calculateRowStart(int n_rows, int threadNum, int blockSize) {
    int rows_per_process = n_rows / blockSize;
    int rem = n_rows % blockSize;
    return (threadNum <= rem) ? (rows_per_process + 1) * threadNum : (rows_per_process + 1) * rem + rows_per_process * (threadNum - rem);;
}

__device__ Matrix generateSubmatrix(Matrix &big_mat, int threadNum, int blockSize) {
    int rows_per_process = big_mat.rows / blockSize;
    int rem = big_mat.rows % blockSize;

    Matrix sub_mat;
    sub_mat.rows = (threadNum < rem) ? (rows_per_process + 1) : rows_per_process;
    sub_mat.cols = big_mat.cols;

    int row_start = calculateRowStart(big_mat.rows, threadNum, blockSize);
    sub_mat.mat = &big_mat.mat[row_start * sub_mat.cols];

    return sub_mat;
}

__global__ void matrixInversion(Matrix m, int blockSize) {
    Matrix local_mat = generateSubmatrix(m, threadIdx.x, blockSize);

    extern __shared__ double pivot[];

    int thread_pivot_row_start = calculateRowStart(m.rows, threadIdx.x, blockSize);
    int thread_pivot_row_end = thread_pivot_row_start + local_mat.rows;
    int n = m.rows;

    for (int i = 0; i < n; ++i) {
        if (thread_pivot_row_start <= i && i < thread_pivot_row_end) {
            double d = getElementDev(local_mat, i - thread_pivot_row_start, i);

            for (int j = 0; j < 2 * n; ++j) {
                double x = getElementDev(local_mat, i - thread_pivot_row_start, j) / d;
                setElementDev(local_mat, i - thread_pivot_row_start, j, x);
            }
        }

        for (int col_block_i = (m.cols - 1) / MAX_SHARED_MEM; col_block_i >= 0; col_block_i--) {
            int col_block_j_start = col_block_i * MAX_SHARED_MEM;
            int col_block_j_end = min((col_block_i + 1) * MAX_SHARED_MEM, m.cols);


            if (thread_pivot_row_start <= i && i < thread_pivot_row_end) {
                for (int j = col_block_j_start; j < col_block_j_end; j++) {
                    pivot[j - col_block_j_start] = getElementDev(local_mat, i - thread_pivot_row_start, j);
                }
            }
            __syncthreads();

            for (int local_i = 0; local_i < local_mat.rows; local_i++) {
                if (thread_pivot_row_start <= i && i < thread_pivot_row_end && local_i == i - thread_pivot_row_start)
                    continue;

                double factor = getElementDev(local_mat, local_i, i);

                for (int j = col_block_j_start; j < col_block_j_end; ++j) {
                    double x = getElementDev(local_mat, local_i, j);
                    double y = x - (pivot[j - col_block_j_start] * factor);
                    setElementDev(local_mat, local_i, j, y);
                }
            }
            __syncthreads();
        }
    }
}

int main(void) {
    Matrix m;
    readMatrix(m);

    size_t size = m.rows * m.cols * sizeof(double);
    Matrix d_m;
    d_m.rows = m.rows;
    d_m.cols = m.cols;
    cudaMalloc((void**)&d_m.mat, size);
    cudaMemcpy(d_m.mat, m.mat, size, cudaMemcpyHostToDevice);

    int blockSize = min(MAX_BLOCK_SIZE, m.rows);
    int sharedMemSize = blockSize * sizeof(double) * 2;

    matrixInversion<<<1, blockSize, sharedMemSize>>>(d_m, blockSize);

    cudaMemcpy(m.mat, d_m.mat, size, cudaMemcpyDeviceToHost);

    printf("%d\n", m.rows);
    printIdent(m);
    cudaFree(d_m.mat);

    cudaDeviceSynchronize();

    return 0;
}

Writing cuda.cu


# Serial Code

In [11]:
%%writefile serial.cpp
#include <iostream>
#include <chrono>
#include <cmath>

using namespace std;

struct Matrix {
    int size;
    double **mat;
};

void allocateMatrix(Matrix &m, int size) {
    m.size = size;
    m.mat = new double*[size];
    for(int i = 0; i < size; ++i) {
        m.mat[i] = new double[2*size];
        for(int j = 0; j < 2*size; ++j) {
            m.mat[i][j] = (j - size == i) ? 1.0 : 0.0;
        }
    }
}

void freeMatrix(Matrix &m) {
    for(int i = 0; i < m.size; ++i)
        delete[] m.mat[i];
    delete[] m.mat;
}

void readMatrix(Matrix &m) {
    int n;
    cin >> n;
    allocateMatrix(m, n);
    for (int i = 0; i < n; ++i)
        for (int j = 0; j < n; ++j)
            cin >> m.mat[i][j];
}

void gaussJordan(Matrix &m) {
    int n = m.size;

    for(int i = 0; i < n; ++i) {
        double d = m.mat[i][i];
        for(int j = 0; j < 2*n; ++j) {
            m.mat[i][j] /= d;
        }

        for(int k = 0; k < n; ++k) {
            if(k != i) {
                double factor = m.mat[k][i];
                for(int j = 0; j < 2*n; ++j) {
                    m.mat[k][j] -= factor * m.mat[i][j];
                }
            }
        }
    }
}

void printMatrix(const Matrix &m) {
    int n = m.size;
    for(int i = 0; i < n; ++i) {
        for(int j = n; j < 2*n; ++j) {
            cout << m.mat[i][j] << " ";
        }
        cout << endl;
    }
}

int main() {
    Matrix m;

    readMatrix(m);
    auto start = std::chrono::high_resolution_clock::now();
    gaussJordan(m);
    auto stop = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = stop - start;

    printMatrix(m);
    cout << "Time taken by function: " << duration.count() << " seconds" << endl;


    freeMatrix(m);

    return 0;
}

Writing serial.cpp


# Compile Programs

In [16]:
!nvcc cuda.cu -o ./bin/cuda
!g++ serial.cpp -o ./bin/serial

# Copy files from GDrive to env

In [9]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/sister-data ./test_case

# Test Case 32

In [17]:
!time ./bin/cuda < ./test_case/32.txt > ./output/output_cuda_32.txt


real	0m0.348s
user	0m0.092s
sys	0m0.127s


In [18]:
!time ./bin/serial < ./test_case/32.txt > ./output/output_serial_32.txt


real	0m0.007s
user	0m0.003s
sys	0m0.002s


# Test Case 64

In [19]:
!time ./bin/cuda < ./test_case/64.txt > ./output/output_cuda_64.txt


real	0m0.159s
user	0m0.036s
sys	0m0.118s


In [21]:
!time ./bin/serial < ./test_case/64.txt > ./output/output_serial_64.txt


real	0m0.012s
user	0m0.009s
sys	0m0.003s


# Test Case 128

In [22]:
!time ./bin/cuda < ./test_case/128.txt > ./output/output_cuda_128.txt


real	0m0.218s
user	0m0.102s
sys	0m0.113s


In [23]:
!time ./bin/serial < ./test_case/128.txt > ./output/output_serial_128.txt


real	0m0.043s
user	0m0.042s
sys	0m0.000s


# Test Case 256

In [24]:
!time ./bin/cuda < ./test_case/256.txt > ./output/output_cuda_256.txt


real	0m0.485s
user	0m0.360s
sys	0m0.117s


In [25]:
!time ./bin/serial < ./test_case/256.txt > ./output/output_serial_256.txt


real	0m0.465s
user	0m0.428s
sys	0m0.005s


# Test Case 512

In [26]:
!time ./bin/cuda < ./test_case/512.txt > ./output/output_cuda_512.txt


real	0m1.329s
user	0m1.198s
sys	0m0.121s


In [27]:
!time ./bin/serial < ./test_case/512.txt > ./output/output_serial_512.txt


real	0m2.011s
user	0m1.965s
sys	0m0.015s


# Test Case 1024

In [28]:
!time ./bin/cuda < ./test_case/1024.txt > ./output/output_cuda_1024.txt


real	0m7.260s
user	0m7.064s
sys	0m0.142s


In [29]:
!time ./bin/serial < ./test_case/1024.txt > ./output/output_serial_1024.txt


real	0m12.962s
user	0m12.796s
sys	0m0.037s


# Test Case 2048

In [30]:
!time ./bin/cuda < ./test_case/2048.txt > ./output/output_cuda_2048.txt


real	0m42.424s
user	0m41.676s
sys	0m0.240s


In [31]:
!time ./bin/serial < ./test_case/2048.txt > ./output/output_serial_2048.txt


real	1m37.220s
user	1m36.294s
sys	0m0.169s
